In [2]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import time
import sys
import os

import seaborn as sns
import matplotlib.dates as mdates
from datetime import datetime, date

# preamble for jupyter notebook and django
import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append("/home/muef/tmv/BasicBrowser/")
    #sys.path.append('/home/leey/tmv/BasicBrowser/')
    dtm_path = "/home/galm/software/dtm/dtm/main"
else:
    # local paths
    sys.path.append('/media/Data/MCC/tmv/BasicBrowser/')
    dtm_path = "/home/finn/dtm/dtm/main"

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
import scoping.models as sm
from parliament.tasks import do_search, run_tm, combine_searches
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import * 
from utils.tm_mgmt import update_topic_scores
from django.db.models import Q, Count, Func, F, Sum, Avg, Value as V
from django.db.models.functions import TruncDate, TruncMonth, TruncYear

In [22]:
# topic labeling

run_id = 2155
no_top_words = 20

labels = {}

df = pd.DataFrame(columns=['label', 'top words', 'score pct', 'score'])

get_input = False

for topic in Topic.objects.filter(run_id = run_id):
    stat = RunStats.objects.get(run_id=run_id)

    print("Topic #{}".format(topic.id))
    
    if stat.method == 'BD':
        
        terms = Term.objects.filter(topicterm__topic=topic).values('title').annotate(
    summed_score = Sum(F('topicterm__score'))).order_by('-summed_score')[:20]
        term_list = [t['title'] for t in terms]
        print(", ".join(term_list))

    else:
        print("Top words: {}".format(topic.top_words))

    if get_input:
        label = input("topic label: ")
        if label:
            labels[topic.id] = label
            topic.title = label
            topic.save()
    else:
        print("topic label: {}".format(topic.title))
    
    print("")
    
    df.loc[topic.id] = {"top words": ", ".join(term_list), "score": topic.score, "label": topic.title, "score pct": 0}

print(labels)
df['score pct'] = 100 * df['score'] / df['score'].sum()
df = df.sort_values(by=['score'], ascending=False)

Topic #178499
sozial, wohnung, eigentum, offent, wohnungsbau, miet, zonenrandgebiet, bund, unternehm, privat, forder, hand, bau, wohn, privatisier, mittel, bundesregier, bauwirtschaft, akti, million
topic label: housing policy

Topic #178474
landwirtschaft, bau, landlich, raum, betrieb, landwirt, agrarpolit, preis, deutsch, grun, minist, verbrauch, brussel, markt, ewg, million, europa, letzt, produktion, zahl
topic label: agriculture

Topic #178478
deutsch, franzos, frankreich, europa, deutschland, behord, vertrag, hoh, regier, national, montanunion, volk, interess, kohl, bundeskanzl, recht, entscheid, art, stahl, schumanplan
topic label: German-French coal plan

Topic #178485
kernkraftwerk, kernenergi, ukrain, technisch, anlag, reaktor, technik, betrieb, bundesrepubl, tschernobyl, nutzung, gefahr, schnell, radioaktiv, friedlich, brut, bau, international, kraftwerk, wissenschaft
topic label: nuclear energy

Topic #178480
energiepolit, kernenergi, ausstieg, bundesregier, kernkraftwerk, 

Topic #178506
berlin, stadt, gebiet, deutsch, haf, bevolker, seit, bundesgebiet, arbeit, lag, betrieb, ministerium, regier, hilf, industri, verfug, haus, teil, wirtschaft, million
topic label: Berlin

Topic #178504
kommun, bund, gemeind, kommunal, nordrheinwestfal, belast, steu, hoh, umsatzsteu, last, gewerbesteu, erhoh, bundesfinanzminist, bundesland, finanzausgleich, million, anteil, grund, zahl, betrag
topic label: financial equalization

Topic #178495
europa, gemeinsam, deutschland, deutsch, staat, gemeinschaft, vertrag, union, bundesregier, bundesrepubl, zusammenarbeit, europas, interess, beid, sowjetunion, national, wirtschaft, partn, regier, verhandl
topic label: europe

Topic #178473
international, entwicklungsland, usa, staat, weltweit, deutsch, dollar, export, amerikan, bundesrepubl, eig, entwicklungspolit, industrieland, vereinigt, japan, weltwirtschaft, abhang, europa, handel, entwickl
topic label: diplomacy & trade

Topic #178483
wissenschaft, forschung, bereich, forder, p

In [23]:
df

,label,top words,score pct,score
178515,stop words,"darauf, darub, natur, richtig, tun, wiss, stell, bereich, lass, gerad, seh, seit, punkt, desweg, darf, namlich, ding, klar, wirklich, eben",15.737580,619234.291928
178481,procedural 1,"bundeskanzl, red, deutsch, damal, sei, wirklich, tag, sagt, letzt, regier, seit, wiss, wort, weiss, recht, partei, deutschland, gar, tun, zeit",6.958758,273809.686006
178498,economic policy 1,"bundesregier, wirtschaft, wichtig, entwickl, hoh, insbesond, deutlich, bereich, stark, ziel, beitrag, seit, entscheid, rund, rahm, dabei, bund, wesent, deutsch, zukunft",5.158223,202963.147750
178513,procedural 2,"gesetz, gesetzentwurf, bundestag, bundesregier, entwurf, bundesrat, fall, vorlieg, zustimm, grund, vorgeseh, verfahr, haus, entscheid, entsprech, begrund, offent, recht, art, zweit",3.622950,142553.991272
178495,europe,"europa, gemeinsam, deutschland, deutsch, staat, gemeinschaft, vertrag, union, bundesregier, bundesrepubl, zusammenarbeit, europas, interess, beid, sowjetunion, national, wirtschaft, partn, regier, verhandl",3.593901,141410.998528
178489,procedural 3,"gesetz, wirtschaft, entwurf, bundesregier, bundesminist, grun, punkt, stimm, spd, gesetzentwurf, zweit, aussprach, ewg, haushaltsausschuss, beschloss, wort, sitzung, link, landwirtschaft, deutsch",3.068980,120756.670825
178497,general policy statements,"freiheit, staat, sozial, volk, gesellschaft, deutsch, burg, leb, demokrat, schul, verantwort, zukunft, demokrati, bildung, generation, famili, recht, regier, arbeit, leistung",2.963721,116614.982983
178478,German-French coal plan,"deutsch, franzos, frankreich, europa, deutschland, behord, vertrag, hoh, regier, national, montanunion, volk, interess, kohl, bundeskanzl, recht, entscheid, art, stahl, schumanplan",2.946700,115945.235466
178484,budget,"million, mittel, haushalt, rund, bund, insgesamt, betrag, hoh, vorgeseh, ausgab, bundeshaushalt, milliard, verfug, zahl, erhoh, gegenub, zuschuss, deutsch, ansatz, haushaltsplan",2.811943,110642.907628
178472,green opposition,"geld, okolog, endlich, regier, wirklich, statt, eigent, wenig, red, kolleginn, merkel, zukunft, warum, tun, lang, gerad, mal, bundesregier, einfach, wort",2.589397,101886.271129


In [24]:
fname = "./plots/topic_analysis_{}/0_topic_list.xlsx".format(run_id)
writer = pd.ExcelWriter(fname, engine='xlsxwriter')

df.to_excel(writer)

In [25]:
pd.set_option('max_colwidth', 500)

fname = "./plots/topic_analysis_{}/0_topic_list.tex".format(run_id)

with open(fname,'w') as tf:
    tf.write(df.to_latex(index=False, columns=["label", "top words", "score pct"],
                         column_format='L{3cm}L{10.5cm}r', float_format="{:0.2f}".format,
                        longtable=True).replace('\n',
                                                '\n\\caption{List of topics with labels, top words and scores (in percent of total score).}
                                                '\n\label{tab:topic-list}\n\small\\\\\n', 1))

In [21]:
# modify single labels:

topic = Topic.objects.get(id=178480)
topic.title = 'nuclear phase out 1'
topic.save()